In [160]:
import numpy as np
import random

In [161]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

In [162]:
# if neg = False, vals will be from 0-1, else -1 - 1
def map_gen(neg=False, mul=1, dim=2, alphabet=alphabet):
    symbols = {}
    for char in alphabet:
        symbols[char] = np.random.rand(dim, dim)
    if neg:
        for char in alphabet:
            with np.nditer(symbols[char], op_flags=['readwrite']) as it:
                for x in it:
                    x[...] = 2 * x
                    if x > 1:
                        x[...] = 1 - x
    if mul != 1:
        for char in alphabet:
            with np.nditer(symbols[char], op_flags=['readwrite']) as it:
                for x in it:
                    x[...] = x * mul
    return symbols

In [163]:
# if neg = False, vals will be from 0-1, else -1 - 1
def map_gen(neg=False, mul=1, dim=2, alphabet=alphabet):
    symbols = {}
    for char in alphabet:
        symbols[char] = np.random.rand(dim, dim)
    if neg:
        for char in alphabet:
            with np.nditer(symbols[char], op_flags=['readwrite']) as it:
                for x in it:
                    x[...] = 2 * x
                    if x > 1:
                        x[...] = 1 - x
    if mul != 1:
        for char in alphabet:
            with np.nditer(symbols[char], op_flags=['readwrite']) as it:
                for x in it:
                    x[...] = x * mul
    return symbols

In [164]:
map_gen(neg=-1, mul=10)

{'a': array([[ 4.65077752, -6.55161835],
        [ 8.96037278, -5.82871237]]),
 'b': array([[ 7.09675344,  6.94840285],
        [ 1.6891166 , -4.57290999]]),
 'c': array([[-2.7514763 , -1.11225249],
        [-6.52597962,  0.24896924]]),
 'd': array([[ 6.02198448,  7.77783184],
        [-8.63101379, -4.65784933]]),
 'e': array([[ 6.72019079,  2.11499154],
        [-8.3703028 , -9.74308821]]),
 'f': array([[-2.56529821, -8.1416633 ],
        [-2.53306937, -8.53189007]]),
 'g': array([[6.73807189, 2.57808649],
        [6.00540538, 3.44439244]]),
 'h': array([[-8.69614688, -5.170066  ],
        [-2.3851048 , -4.51805477]]),
 'i': array([[-1.53920434, -6.52305718],
        [ 6.52675453,  0.31645747]]),
 'j': array([[ 3.40212298, -8.11884275],
        [ 1.9794458 , -5.05272308]]),
 'k': array([[ 4.48721277,  1.49620485],
        [-1.53847296,  3.7234007 ]]),
 'l': array([[ 4.58305436, -7.30279528],
        [ 6.32325688, -9.32414772]]),
 'm': array([[-1.74596024, -8.07619775],
        [ 1.222

In [165]:
def wordMult(word):
  """Converts a word into its matrix form, by multiplying the symbol matrices."""
  if(len(word) == 1): return alphabet.get(word)
  mp = np.dot(alphabet.get(word[len(word)-2]), alphabet.get(word[len(word)-1]))
  if(len(word) == 2): 
    return mp
  for i in range(len(word)-2):
    mp = np.dot(alphabet.get(word[len(word)-i-3]), mp)
  return mp

def getTrace(arr):
  """Get the trace of a matrix (sum of the diagonal elements)."""
  return np.trace(arr)

def checkDensityValidity(Pl, Pr):
  """Check if the density matrices, Pl and Pr, are valid. The trace of their product must be 1."""
  mp = np.dot(Pl, Pr)
  return (getTrace(mp) == 1)

def phi(M, Pl, Pr):
  """Get the estimated frequency of a word. tr(Pl M Pr M*)"""
  M_cross = np.asmatrix(M).getH() # gets complex conjugate transpose
  mp = np.dot(Pr, M_cross)
  mp = np.dot(M, mp)
  mp = np.dot(Pl, M)
  # used to absolute value, but that was because our symbol matrices had negative values :D
  return getTrace(mp)
    
def grabValuesRandom(iterations, high, target):
  bestError = 1000
  bestPl = None
  bestPr = None
  bestEstimates = {}
  iter = 0
  for a in range(iterations):
    a1 = (random.random()*100) % high
    for b in range(iterations):
      b1 = (random.random()*100) % high
      for c in range(iterations):
        c1 = (random.random()*100) % high
        for d in range(iterations):
          d1 = (random.random()*100) % high
          for e in range(iterations):
            e1 = (random.random()*100) % high
            for f in range(iterations):
              f1 = (random.random()*100) % high
              for g in range(iterations):
                g1 = (random.random()*100) % high
                h=(1-(a1*e1+b1*g1+c1*f1))/d1
                m1 = np.array([[a1, b1],[c1, d1]])
                m2 = np.array([[e1, f1],[g1, h]])
                iter += 1
                if checkDensityValidity(m1, m2):
                  totalError = 0
                  estimates = {}
                  for z in target.keys():
                    totalError += ((phi(wordMult(z), m1, m2)-target[z])**2)
                    estimates[z] = phi(wordMult(z), m1, m2)
                    
                  if (totalError < bestError):
                    bestEstimates = estimates
                    bestError = totalError
                    bestPl = m1
                    bestPr = m2 

  return bestError, bestEstimates, bestPl, bestPr
  
targetFrequencies = {"art": 0.15, "rat": 0.15, "at": 0.25, "a": 0.33, "tar": 0.05}
bestError = 1000
bestMap = None
bestEstimates = {}
finalPl = None
finalPr = None
for i in range(10):
  print(f"Iteration: {i}.")
  alphabet = map_gen(alphabet="art")
  error, d, Pl, Pr = grabValuesRandom(6, 1, targetFrequencies)
  if (error < bestError):
    bestEstimates = d
    bestError = error
    bestMap = alphabet
    print(f"Current best error sum: {bestError}.")

for word in bestEstimates:
  print(f"Word: {word}.")
  print(f"Real freq: {targetFrequencies[word]}.")
  print(f"Calculated freq: {bestEstimates[word]}.")
  print(f"Deviance: {abs(targetFrequencies[word]-bestEstimates[word])}.\n")

Iteration: 0.
Current best error sum: 0.09500130077762421.
Iteration: 1.
Current best error sum: 0.0844785294791306.
Iteration: 2.
Current best error sum: 0.010281599904520558.
Iteration: 3.
Iteration: 4.
Iteration: 5.
Iteration: 6.
Iteration: 7.
Iteration: 8.
Iteration: 9.
Word: art.
Real freq: 0.15.
Calculated freq: 0.1632346763080731.
Deviance: 0.013234676308073096.

Word: rat.
Real freq: 0.15.
Calculated freq: 0.17676484894813915.
Deviance: 0.02676484894813916.

Word: at.
Real freq: 0.25.
Calculated freq: 0.16974184395360764.
Deviance: 0.08025815604639236.

Word: a.
Real freq: 0.33.
Calculated freq: 0.3446757614749485.
Deviance: 0.014675761474948479.

Word: tar.
Real freq: 0.05.
Calculated freq: 0.1022813209615799.
Deviance: 0.05228132096157989.

